# **Modelo de Recomendação**
Nesse notebok será trabalhado no modelo de recomendação de produtos com dados de usuários de Olist, que foram selecionados e gerados a partir do notebook **analise_exploratoria_olist.ipnyb** (recomendo a leitura). Além de treinarmos o modelo para recomendação, vamos validar sua qualidade de previsão, testar dados absolutos e normalizados, e alguns modelos para entender qual seria a melhor opção para essa base de dados.

## **Resumo do Modelo**
AAAAAAAAAAAAAAA

In [41]:
import random
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [42]:
dataset = pd.read_excel("dataset.xlsx")
dataset.head()

,order_id,customer_id,customer_unique_id,customer_city,customer_state,product_id,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,87285b34884572647811a353c7ac498a,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,4
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,barreiras,BA,595fac2a385ac33a80bd5114aec74eb8,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,4
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,3a653a41f6f9fc3d2a113cf8398680e8,vianopolis,GO,aa4383b373c6aca5d8797843e5594415,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,5
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,7c142cf63193a1473d2e66489a9ae977,sao goncalo do amarante,RN,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,27.20,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0,5
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,72632f0f9dd73dfee390c9b22eb56dd6,santo andre,SP,65266b2da20d04dbe00c5c2d3bb7859e,19.90,8.72,papelaria,38.0,316.0,4.0,250.0,51.0,15.0,15.0,5


In [43]:
#Selecionando características de produtos e usuários
produto_features = ["price", "freight_value", "product_category_name", 
                    "product_name_lenght", "product_description_lenght", "product_photos_qty",
                    "product_weight_g", "product_length_cm", "product_height_cm",
                    "product_width_cm"]
cliente_features = ["customer_city", "customer_state"]

In [44]:
dataset["product_category_name"].fillna("unknown", inplace=True)

C:\Users\Computador\AppData\Local\Temp\ipykernel_8948\1744511927.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["product_category_name"].fillna("unknown", inplace=True)


In [45]:
#Apenas teste por enquanto
dataset = dataset.sample(n=2000, random_state=42)

**Gerando uma amostra negativa**
No dataset existe apenas dados de compras realizadas, como a intenção é desenvolver um modelo de classificação para realizar a recoemndação de produtos, será necessário criar uma amostra aleatória de compras não realizadas para auxiliar o modelo no entendimento dos dados.

**Amostragem Negativa Simples**

In [46]:
positivos = dataset[["customer_unique_id", "product_id"]].drop_duplicates()
positivos['target'] = 1

clientes = dataset["customer_unique_id"].unique()
produtos = dataset["product_id"].unique()

#Gerando a amostra negativa
negativos = []

for _ in range(len(positivos)):
    c = random.choice(clientes)
    p = random.choice(produtos)
    if not ((positivos["customer_unique_id"] == c) & (positivos["product_id"]) == p).any():
        negativos.append((c, p, 0))
negativos = pd.DataFrame(negativos, columns=["customer_unique_id", "product_id", "target"])

#Juntando interacoes
interacoes = pd.concat([positivos, negativos])

#Juntando atributos de cliente e produtos
dados_ns = interacoes.merge(dataset.drop_duplicates("customer_unique_id")[["customer_unique_id"] + cliente_features], 
                            on="customer_unique_id", how="left")
dados_ns = dados_ns.merge(dataset.drop_duplicates("product_id")[["product_id"] + produto_features], 
                          on="product_id", how="left")

**Amostragem Negativa por Similaridade**

In [47]:
def gerar_negativos_similares(df, n_amostras=1):
    negativos = []
    for cid in df['customer_unique_id'].unique():
        comprados = df[df['customer_unique_id'] == cid]['product_category_name'].unique()
        produtos_similares = df[~df['product_category_name'].isin(comprados)]['product_id'].unique()

        for _ in range(n_amostras):
            p = random.choice(produtos_similares)
            negativos.append((cid, p, 0))
    
    return pd.DataFrame(negativos, columns=['customer_unique_id', 'product_id', 'target'])


In [48]:
dados_nsimilirdade = gerar_negativos_similares(dataset)

**Pré-processamento**

In [49]:
dados = dados_ns.copy()

In [50]:
X = dados[cliente_features + produto_features]
y = dados['target']

num_features = list(dados.select_dtypes("float").columns)
cat_features = list(dados.select_dtypes("object").columns)
cat_features.remove("customer_unique_id")
cat_features.remove("product_id")

In [51]:
dados_nulos = dados[num_features].isnull().sum()
poucos_nulos = dados_nulos[dados_nulos <= 50].index.tolist()
muitos_nulos = dados_nulos[dados_nulos > 50].index.tolist()

In [52]:
num_pipeline_poucos = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

num_pipeline_muitos = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [53]:
preprocessor = ColumnTransformer([
    ('num_poucos', num_pipeline_poucos, poucos_nulos),
    ('num_muitos', num_pipeline_muitos, muitos_nulos),
    ('cat', cat_pipeline, cat_features)
])

**Random Forest Classifier**

In [54]:
#Modelo
modelo = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classicador', RandomForestClassifier(n_estimators=100))
])

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

modelo.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num_poucos',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['price', 'freight_value',
                                                   'product_weight_g',
                                                   'product_length_cm',
                                                   'product_height_cm',
                                                   'product_width_cm']),
                                                 ('num_muitos',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['product_name_lenght',
                                                   'product_description_lenght',
                                                   'product_photos_qty']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['customer_city',
                                                   'customer_state',
                                                   'product_category_name'])])),
                ('classicador', RandomForestClassifier())])

In [56]:
probas = modelo.predict_proba(X_test)[:, 1]

# **Recomendação**

In [60]:
cliente_id = dataset["customer_unique_id"][8027]

In [61]:
produtos_comprados = dados[dados['customer_unique_id'] == cliente_id]['product_id'].unique()
todos_produtos = dados['product_id'].unique()

produtos_nao_comprados = [p for p in todos_produtos if p not in produtos_comprados]

In [62]:
# Pegamos as informações do cliente
cliente_info = dados[dados['customer_unique_id'] == cliente_id].iloc[0]

# Criar novas linhas combinando o cliente com os produtos não comprados
recomendacoes = []

for produto_id in produtos_nao_comprados:
    prod_info = dados[dados['product_id'] == produto_id].iloc[0]

    linha = {
        'customer_unique_id': cliente_id,
        'customer_state': cliente_info['customer_state'],
        'customer_city': cliente_info['customer_city'],
        'product_id': produto_id,
        'product_category_name': prod_info['product_category_name'],
        'price': prod_info['price'],
        'freight_value': prod_info['freight_value'],
        'product_name_lenght': prod_info['product_name_lenght'],
        'product_description_lenght': prod_info['product_description_lenght'],
        'product_photos_qty': prod_info['product_photos_qty'],
        'product_weight_g': prod_info['product_weight_g'],
        'product_length_cm': prod_info['product_length_cm'],
        'product_height_cm': prod_info['product_height_cm'],
        'product_width_cm': prod_info['product_width_cm']
    }

    recomendacoes.append(linha)

df_recomendacoes = pd.DataFrame(recomendacoes)


In [63]:
X_recomendacao = df_recomendacoes[cliente_features + produto_features]
scores = modelo.predict_proba(X_recomendacao)[:, 1]

df_recomendacoes['score'] = scores


In [64]:
top_n = 5
top_recomendacoes = df_recomendacoes.sort_values(by='score', ascending=False).head(top_n)

print(top_recomendacoes[['product_id', 'product_category_name', 'score']])


                           product_id product_category_name  score
54   aca2eb7d00ea1a7b8ebd4e68314663af      moveis_decoracao   0.96
454  11fd9d7ba552e1d28872e008acf1e050            cool_stuff   0.94
253  53b36df67ebb7c41585e8d54d6772e08    relogios_presentes   0.93
243  389d119b48cf3043d311335e499d9c6b    ferramentas_jardim   0.92
435  d285360f29ac7fd97640bf0baef03de0    relogios_presentes   0.92
